In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import datetime, timedelta
import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [48]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

df

Today's date: 2025-11-25


,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,STL_BLK,PR,PA,RA,PRA
0,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Christian Braun,SG,1,19.33,2,5,0.400,0,1,0.000,1,2,0.5,1,2,3,2,0,1,1,1,5,3.9,5,0,0,8.5,1,8,7,5,10
1,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Kentavious Caldwell-Pope,SG,1,36.25,8,12,0.667,2,3,0.667,2,2,1.0,1,1,2,1,3,1,3,5,20,15.2,10,0,0,19.0,4,22,21,3,23
2,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Collin Gillespie,PG,1,0.72,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
3,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Aaron Gordon,PF,1,34.98,7,11,0.636,1,2,0.500,0,0,0.0,2,5,7,5,2,1,0,0,15,19.2,6,0,0,26.0,3,22,20,12,27
4,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Justin Holiday,SG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6832,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Maxime Raynaud,C,1,13.92,0,4,0.000,0,1,0.000,1,2,0.5,1,1,2,0,0,0,0,3,1,-2.4,-5,0,0,2.5,0,3,1,2,3
6833,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Dario Saric,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
6834,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Doug McDermott,SF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
6835,2025,20251124_MIN_SAC,2025-11-24,SAC,Home,MIN,Away,Keegan Murray,PF,1,44.40,10,19,0.526,3,5,0.600,3,3,1.0,2,12,14,2,1,1,0,3,26,23.6,6,1,0,35.5,2,40,28,16,42


In [47]:
%run ./common_utils.ipynb

In [32]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
263,2025-11-26,5:00p,Detroit Pistons,DET,Boston Celtics,BOS,TD Garden,0,0,0
264,2025-11-26,7:00p,New York Knicks,NYK,Charlotte Hornets,CHO,Spectrum Center,0,0,0
265,2025-11-26,7:30p,Milwaukee Bucks,MIL,Miami Heat,MIA,Kaseya Center,0,0,0
266,2025-11-26,7:30p,Minnesota Timberwolves,MIN,Oklahoma City Thunder,OKC,Paycom Center,0,0,0
267,2025-11-26,7:30p,Indiana Pacers,IND,Toronto Raptors,TOR,Scotiabank Arena,0,0,0
268,2025-11-26,8:00p,Memphis Grizzlies,MEM,New Orleans Pelicans,NOP,Smoothie King Center,0,0,0
269,2025-11-26,10:00p,Houston Rockets,HOU,Golden State Warriors,GSW,Chase Center,0,0,0
270,2025-11-26,10:00p,San Antonio Spurs,SAS,Portland Trail Blazers,POR,Moda Center,0,0,0
271,2025-11-26,10:00p,Phoenix Suns,PHO,Sacramento Kings,SAC,Golden 1 Center,0,0,0


In [33]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = 2025 AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-11-25_11PM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,SAS,Victor Wembanyama,C,37.833333,2,2025-11-26,Out,Injury/Illness - Left Calf; Strain
1,SAC,Domantas Sabonis,C,25.772727,5,2025-11-26,Out,Injury/Illness - Left Knee; Partial Meniscus Tear
2,POR,Jrue Holiday,PG,24.125000,11,2025-11-26,Out,Injury/Illness - Right Calf; Strain
3,SAS,Stephon Castle,PG,23.115385,13,2025-11-26,Out,Injury/Illness - Left Hip Flexor; Strain
4,HOU,Kevin Durant,SF,21.576923,13,2025-11-26,Out,Personal Reasons
5,PHO,Grayson Allen,SG,20.961538,9,2025-11-26,Out,Injury/Illness - Right Quadriceps; Contusion
6,NYK,OG Anunoby,PF,20.375000,14,2025-11-26,Out,Injury/Illness - Left Hamstring; Strain
7,TOR,RJ Barrett,SF,19.764706,18,2025-11-26,Out,Injury/Illness - Right Knee; Sprain
8,MEM,Ja Morant,PG,19.125000,21,2025-11-26,Out,Injury/Illness - Right Calf; Strain
9,BOS,Neemias Queta,C,17.705882,27,2025-11-26,Out,Injury/Illness - Left Ankle; Sprain


In [35]:
def get_game_odds():

    dk_tm_mapping = {
                    "ATL Hawks": "ATL", "BKN Nets": "BRK", "BOS Celtics": "BOS", "CHA Hornets": "CHO", "CHI Bulls": "CHI",
                    "CLE Cavaliers": "CLE", "DAL Mavericks": "DAL", "DEN Nuggets": "DEN", "DET Pistons": "DET", 
                    "GS Warriors": "GSW", "HOU Rockets": "HOU", "IND Pacers": "IND", "LA Clippers": "LAC", 
                    "LA Lakers": "LAL", "MEM Grizzlies": "MEM", "MIA Heat": "MIA", "MIL Bucks": "MIL", "MIN Timberwolves": "MIN",
                    "NO Pelicans": "NOP", "NY Knicks": "NYK", "OKC Thunder": "OKC", "ORL Magic": "ORL", "PHI 76ers": "PHI",
                    "PHO Suns": "PHO", "POR Trail Blazers": "POR", "SA Spurs": "SAS", "SAC Kings": "SAC", "TOR Raptors": "TOR",
                    "UTA Jazz": "UTA", "WAS Wizards": "WAS", "GSW Warriors": "GSW", "LAL Lakers": "LAL", "NOP Pelicans": "NOP",
                    "NYK Knicks": "NYK", "PHX Suns": "PHO", "SAS Spurs": "SAS"
                    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }
    response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C4511&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%274511%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%274511%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
    if response.status_code != 200:
        raise Exception('Bad Request')

    teams = []
    spreads = []
    totals = []
    for i in range(len(response.json()['selections'])):    
        if 'HC' in response.json()['selections'][i]['id']:
            teams.append(response.json()['selections'][i]['label'])
            spreads.append(response.json()['selections'][i]['points'])
        elif 'OU' in response.json()['selections'][i]['id']:
            teams.append(response.json()['selections'][i]['label'])
            totals.append(response.json()['selections'][i]['points'])
        else:
            continue

    teams = [x for x in teams if x != "Over"]
    teams = [x for x in teams if x != "Under"]

    df_tm_bets = pd.DataFrame({"Team": teams, "Spread": spreads, "Total": totals})
    for key, value in dk_tm_mapping.items():
        df_tm_bets['Team'] = np.where(df_tm_bets.Team == key, value, df_tm_bets.Team)
    
    return df_tm_bets

def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines = pd.merge(df_lines, get_game_odds(), on='Team')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,STL_line,BLK_line,STL_BLK_line,TPM_line,PA_line,PR_line,RA_line,PRA_line,Spread,Total
0,2025-11-26,MIL,AJ Green,SG,9.5,NaN,2.5,NaN,NaN,NaN,2.5,NaN,11.5,NaN,NaN,6.5,236.5
1,2025-11-26,OKC,Ajay Mitchell,SG,16.5,3.5,3.5,NaN,NaN,NaN,1.5,20.5,19.5,7.5,23.5,-8.5,226.5
2,2025-11-26,OKC,Alex Caruso,SG,7.5,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,-8.5,226.5
3,2025-11-26,HOU,Alperen Sengun,C,22.5,6.5,10.5,NaN,NaN,NaN,NaN,29.5,33.5,17.5,39.5,1.5,225.5
4,2025-11-26,HOU,Amen Thompson,SF,19.5,4.5,7.5,NaN,NaN,NaN,NaN,24.5,26.5,12.5,31.5,1.5,225.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2025-11-26,MIA,Tyler Herro,SG,18.5,3.5,4.5,NaN,NaN,NaN,1.5,22.5,23.5,8.5,27.5,-6.5,236.5
80,2025-11-26,MEM,Vince Williams Jr.,SG,10.5,6.5,5.5,NaN,NaN,NaN,1.5,16.5,15.5,11.5,22.5,-1.5,234.5
81,2025-11-26,NOP,Yves Missi,C,7.5,NaN,6.5,NaN,NaN,NaN,NaN,NaN,14.5,NaN,NaN,1.5,234.5
82,2025-11-26,MEM,Zach Edey,C,12.5,1.5,8.5,NaN,NaN,NaN,NaN,NaN,20.5,9.5,NaN,-1.5,234.5


In [38]:
def pick_finder(stat, collect=False):
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        df_picks = con.execute(f"""SELECT df_picks.*, df_lines.{stat}_line, Spread, Total FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   WHERE {stat}_line IS NOT NULL
                                   ORDER BY Def_Rk DESC""").fetchdf()
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [39]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,Spread,Total
0,MIA,0,Davion Mitchell,10.277778,11.0,5.571429,22,MIL,23.250000,29.8,28,8.5,-6.5,236.5
1,NOP,0,Jeremiah Fears,15.388889,17.4,NaN,15,MEM,22.928571,21.8,27,13.5,1.5,234.5
2,DET,0,Cade Cunningham,27.142857,30.0,19.750000,4,BOS,22.692308,20.8,24,25.5,-2.5,229.5
3,TOR,0,Immanuel Quickley,15.500000,14.8,12.600000,14,IND,22.125000,17.6,21,16.5,-9.5,234.5
4,MIL,0,Ryan Rollins,18.166667,20.0,3.800000,8,MIA,22.083333,19.0,20,18.5,6.5,236.5
5,HOU,0,Reed Sheppard,13.428571,15.0,0.833333,17,GSW,21.000000,19.8,19,14.5,1.5,225.5
6,OKC,0,Shai Gilgeous-Alexander,32.058824,31.4,33.307692,1,MIN,20.333333,13.4,14,30.5,-8.5,226.5
7,NYK,0,Jalen Brunson,28.285714,30.2,25.166667,3,CHO,19.882353,18.6,11,28.5,-6.5,238.5
8,IND,0,Andrew Nembhard,18.300000,19.4,11.333333,7,TOR,19.583333,18.0,10,16.5,9.5,234.5
9,BOS,0,Payton Pritchard,16.647059,21.6,19.500000,13,DET,18.800000,18.2,7,16.5,2.5,229.5


SG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,Spread,Total
0,MEM,0,Jaylen Wells,10.333333,12.2,10.800000,28,NOP,18.750000,19.8,26,13.5,-1.5,234.5
1,MEM,0,Vince Williams Jr.,8.833333,8.4,5.666667,33,NOP,18.750000,19.8,26,10.5,-1.5,234.5
2,MEM,0,Kentavious Caldwell-Pope,7.941176,6.0,11.333333,40,NOP,18.750000,19.8,26,7.5,-1.5,234.5
3,MEM,0,Cedric Coward,13.777778,12.4,NaN,19,NOP,18.750000,19.8,26,13.5,-1.5,234.5
4,MIA,0,Tyler Herro,24.000000,24.0,22.166667,4,MIL,17.857143,21.6,21,18.5,-6.5,236.5
5,MIA,0,Pelle Larsson,10.764706,10.8,3.750000,25,MIL,17.857143,21.6,21,8.5,-6.5,236.5
6,MIA,0,Norman Powell,25.428571,26.6,NaN,3,MIL,17.857143,21.6,21,19.5,-6.5,236.5
7,MIL,0,Gary Trent Jr.,10.222222,11.2,13.625000,29,MIA,17.428571,16.0,18,10.5,6.5,236.5
8,NYK,0,Miles McBride,10.833333,12.6,7.428571,23,CHO,16.937500,16.8,13,11.5,-6.5,238.5
9,NYK,0,Jordan Clarkson,10.812500,14.6,NaN,24,CHO,16.937500,16.8,13,12.5,-6.5,238.5


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,Spread,Total


PF


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,Spread,Total


C


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,Spread,Total


In [41]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Spread, Total, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk, Spread, Total) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-11-26
../tables/2025/parlay_stats.csv saved!


,Date,Team,B2B,Spread,Total,Player,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line
0,2025-11-26,MIA,0,-6.5,236.5,Davion Mitchell,MIL,10.277778,11.0,5.571429,23.250000,29.8,8.5,7.333333,6.6,3.285714,6.437500,8.4,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.611111,17.6,8.857143,29.687500,38.2,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-26,NOP,0,1.5,234.5,Jeremiah Fears,MEM,15.388889,17.4,NaN,22.928571,21.8,13.5,2.882353,2.2,NaN,5.000000,5.0,2.5,3.470588,3.8,NaN,4.928571,4.0,3.5,18.666667,21.2,NaN,27.857143,25.8,17.5,18.111111,19.6,NaN,27.928571,26.8,16.5,6.000000,6.0,NaN,9.928571,9.0,6.5,21.388889,23.4,NaN,32.857143,30.8,20.5,2.000000,2.0,NaN,3.428571,3.4,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-26,DET,0,-2.5,229.5,Cade Cunningham,BOS,27.142857,30.0,19.750000,22.692308,20.8,25.5,9.642857,9.6,8.250000,6.307692,6.2,8.5,6.000000,8.2,5.250000,4.692308,5.8,5.5,33.142857,38.2,25.0,27.384615,26.6,31.5,36.785714,39.6,28.000000,29.000000,27.0,34.5,15.642857,17.8,13.5,11.000000,12.0,14.5,42.785714,47.8,33.25,33.692308,32.8,40.5,1.923077,1.8,1.625000,2.923077,2.8,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-11-26,TOR,0,-9.5,234.5,Immanuel Quickley,IND,15.500000,14.8,12.600000,22.125000,17.6,16.5,6.055556,5.6,4.400000,6.000000,6.2,5.5,4.777778,6.0,3.400000,5.875000,5.2,3.5,20.277778,20.8,16.0,28.000000,22.8,20.5,21.555556,20.4,17.000000,28.125000,23.8,22.5,10.833333,11.6,7.8,11.875000,11.4,10.5,26.333333,26.4,20.40,34.000000,29.0,26.5,2.750000,2.8,1.400000,1.500000,2.0,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-11-26,MIL,0,6.5,236.5,Ryan Rollins,MIA,18.166667,20.0,3.800000,22.083333,19.0,18.5,6.000000,7.2,1.000000,7.666667,8.0,6.5,4.235294,4.0,0.400000,7.333333,6.8,4.5,22.166667,24.0,4.2,29.416667,25.8,22.5,24.166667,27.2,4.800000,29.750000,27.0,24.5,10.000000,11.2,1.4,15.000000,14.8,10.5,28.166667,31.2,5.20,37.083333,33.8,29.5,2.800000,3.6,0.600000,2.750000,2.4,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2025-11-26,IND,0,9.5,234.5,T.J. McConnell,TOR,NaN,NaN,NaN,NaN,NaN,NaN,3.714286,4.6,5.285714,5.583333,6.8,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,2025-11-26,IND,0,9.5,234.5,Ben Sheppard,TOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.294118,3.4,1.875000,3.789474,4.2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.555556,1.2,0.750000,2.052632,2.4,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2025-11-26,IND,0,9.5,234.5,Jarace Walker,TOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.117647,5.0,2.285714,7.111111,5.2,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na